In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import feather
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

In [ ]:
# Load environment variables to get local datasets path
from dotenv import load_dotenv
load_dotenv()
data_dir = os.environ.get('datasets_path')

In [ ]:
df_meta = pd.read_feather('df_meta.fth')